# Problème de Poisson, méthodes itératives "basiques" et le gradient conjugué

Dans ce travail, on se propose de comparer les différentes méthodes de résolution itératives "basiques" avec l'algorithme du gradient conjugué pour la résolution d'un système linéaire dont la matrice est symétrique définie positive. Plus précisément, on va comparer: la méthode de Jacobi, la méthode de Gauss-Siedel et la méthode de plus profonde descente avec le gradient conjugué. 

## 1. Un petit problème pour commencer (et visualiser les choses)

On va débuter avec un problème de taille $2\times 2$ afin de tester les méthodes et voir leur comportement. Pour cela, on va considérer le problème suivant:

$$ Ax=b,$$

où $A = \begin{pmatrix}3 & 2 \\ 2 & 6 \end{pmatrix}$ et $ b = \begin{pmatrix} 2 \\ -8 \end{pmatrix}$. On remarque que la matrice $A$ est symétrique définie positive et que la solution $x$ est aussi la solution du problème de minimisation dans $\mathbb{R}^2$

$$ x = \mbox{argmin}_{y\in\mathbb{R}^2}\frac12 \langle y,Ay\rangle - \langle b,y\rangle. $$

On se propose tout d'abord d'implémenter les méthodes de Jacobi et Gauss-Siedel pour la résolution du système linéaire  et méthode de plus profonde descente pour le problème de minimisation. On veillera à ce que les méthodes soient appliquées via des fonctions qui prendront en entrée une matrice $A\in M_{n,n}(\mathbb{R})$ (taille quelconque), un vecteur $b\in\mathbb{R}^n$, un point de départ $x_0\in\mathbb{R}^n$ et une tolérance pour le critère d'arrêt. On basera ce dernier sur la norme euclidienne de la différence entre $2$ itérés (sauf pour le gradient conjugué). Finalement, on pourra aussi désigner une variable d'entrée pour la méthode de Gauss-Siedel afin d'indiquer la décomposition choisie (triangulaire inférieure ou supérieure). Finalement, il sera préférable (dans un premier temps) de donner en sortie l'ensemble des points calculés par les méthodes afin d'afficher leur évolution.

### 1.0. Chargement des librairies classiques pour le calcul scientifique et rappels

In [ ]:
import numpy as np
import numpy.linalg as npl
import scipy.linalg as spl
import matplotlib.pyplot as plt
%matplotlib inline

# Affichage pratique de la fonction f(x) = 1/2*<x,Ax> - <b,x>
def affichage(P) :
    Nx = 1000
    Ny = 1000
    xmin = np.min(P[:,0]) - 0.5
    xmax = np.max(P[:,0]) + 0.5
    ymin = np.min(P[:,1]) - 0.5
    ymax = np.max(P[:,1]) + 0.5
    x = np.linspace(xmin,xmax,Nx)
    y = np.linspace(ymin,ymax,Ny)
    X, Y = np.meshgrid(x, y)
    Z= 0.5*(3*X**2 + 4*X*Y + 6*Y**2) - 2*X + 8*Y
    V = [k/10.*np.max(Z) + (1-k/10.)*np.min(Z) for k in range(11)]
    CS=plt.contour(X, Y, Z, V,colors='k')
    plt.scatter(P[:,0], P[:,1],marker='o')
    plt.clabel(CS, inline=1, fontsize=10)
    plt.show()
P = np.array([[0,0],[-0.5,0.5],[0.5,-0.5],[1,-1]])
affichage(P)

# Résolution d'un système linéaire
A = np.array([[3,2],[2,6]])
b = np.array([2,-8])
x = npl.solve(A,b)

# Résolution d'un système triangulaire inférieur
T = np.array([[3,0],[2,6]])
x = spl.solve_triangular(T,b,lower = True)

### 1.1. Implémentation de la méthode de Jacobi et Gauss-Siedel

In [ ]:
def Jacobi(A,b,x0 = None,tol = 1e-10):
    if x0 is None:
        x0 = np.zeros(len(b))
    x = np.copy(x0)
    x_tmp = np.copy(x)
    delta = 2*tol
    M = np.diag(np.diag(A),0)
    N = M - A
    M_inv = np.diag(1./np.diag(A),0)
    z = np.dot(M_inv,b)
    Xiter = [x0]
    while delta > tol:
        x_tmp = np.copy(x)
        x = np.dot(M_inv,np.dot(N,x)) + z
        delta = npl.norm(x-x_tmp)
        Xiter += [x]
    return np.array(Xiter)

def Gauss_Siedel(A,b,x0 = None,tol = 1e-10):
    if x0 is None:
        x0 = np.zeros(len(b))
    x = np.copy(x0)
    x_tmp = np.copy(x)
    delta = 2*tol
    M = np.triu(A,0)
    N = M - A
    z = spl.solve_triangular(M,b, lower = False)
    Xiter = [x0]
    while delta > tol:
        x_tmp = np.copy(x)
        x = spl.solve_triangular(M,np.dot(N,x), lower = False) + z
        delta = npl.norm(x-x_tmp)
        Xiter += [x]
    return np.array(Xiter)

### 1.2. Implémentation de la méthode de plus profonde descente

On rappelle que la méthode de plus profonde descente est un algorithme de type gradient dont le pas est optimisé. La méthode s'écrit donc sous la forme

$$ \left\{\begin{array}{ll} x_0\in\mathbb{R}^n, \\ x_{k+1} = x_k - s_k \nabla f(x_k).  \end{array}\right. $$

où $f(x) = \frac12 \langle x,Ax\rangle - \langle b,x\rangle$ et donc $- \nabla f(x_k) = b - Ax_k = r_k$ (qui correspond donc au résidu associé à $x_k$). À chaque itération, $s_k$ est choisit tel que

$$ s_k = \mbox{argmin}_{s\in\mathbb{R}} f(x_k - s \nabla f(x_k)). $$

> **A faire :** Montrer, en dérivant l'expression $f(x_k - s \nabla f(x_k))$ par rapport à $s$, que $s_k$ est explicitement donné par la formule
$$ s_k = \frac{\|r_k\|_2^2}{\langle A r_k,r_k\rangle}. $$
En déduire que les directions de descentes sont orthogonales une à une, c'est-à-dire que
$$ <r_{k+1},r_k> = 0.$$

On en déduit donc que la méthode se décompose comme

$$ \left\{\begin{array}{ll} x_0\in\mathbb{R}^n, \\ r_k = b - Ax_k, \\ s_k = \frac{\|r_k\|_2^2}{\langle A r_k,r_k\rangle}, \\ x_{k+1} = x_k + s_k r_k.  \end{array}\right. $$



In [ ]:
def Steepest_descent(A,b,x0 = None,tol = 1e-10):
    if x0 is None:
        x0 = np.zeros(len(b))
    x = np.copy(x0)
    x_tmp = np.copy(x)
    delta = 2*tol
    Xiter = [x0]
    while delta>tol:
        x_tmp = np.copy(x)
        r = b-np.dot(A,x)
        s = float(npl.norm(r)**2)/np.dot(np.dot(A,r),r)
        x = x + s*r
        delta = npl.norm(x-x_tmp)
        Xiter += [x]
    return np.array(Xiter)

### 1.3. Implémentation du gradient conjugué

L'idée pour améliorer la convergence de la méthode de la plus profonde descente consiste à construire des directions de descentes qui ne sont plus simplement orthogonales une à une mais $A$-conjuguées. On aboutit alors à l'algorithme du gradient conjugué que l'on a vu en cours (sous une approche différente). Le fait d'avoir une famille de direction $A$-conjuguées va dramatiquement améliorer la convergence de la méthode de gradient. Le critère d'arrêt sera, cette fois, basé sur la norme euclidienne du résidu.

In [ ]:
def Conjugate_Gradient(A,b,x0 = None,tol = 1e-10):
    if x0 is None:
        x0 = np.zeros(len(b))
    x = np.copy(x0)
    x_tmp = np.copy(x)
    delta = 2*tol
    Xiter = [x0]
    r = b-np.dot(A,x)
    r_tmp = np.copy(r)
    d = np.copy(r)
    while npl.norm(r)>tol:
        r_tmp = np.copy(r)
        t = float(npl.norm(r)**2)/np.dot(np.dot(A,d),d)
        x = x + t*d
        r = r - t*np.dot(A,d)
        if npl.norm(r)>1e-15:
            s = float(npl.norm(r)**2)/npl.norm(r_tmp)**2
            d = r + s*d
        Xiter += [x]
    return np.array(Xiter)

### 1.4. Comparaison du comportement des différentes méthodes

> **A faire :** À l'aide de la fonction d'affichage que l'on fournit, tester les différentes méthodes et commenter les résultats obtenus pour différents points initiaux. On pourra par exemple prendre comme points de départ:
- x1 = [0,0]
- x2 = [2,2]

In [ ]:
print "TEST: x0 = [0,0]\n"
xini = np.array([0,0])
XJacobi = Jacobi(A,b,xini)
print "Méthode de Jacobi (avec",len(XJacobi[:,0])-1,"itérations):"
affichage(XJacobi)
XGauss_Siedel = Gauss_Siedel(A,b,xini)
print "Méthode de Gauss-Siedel (avec",len(XGauss_Siedel[:,0])-1,"itérations):"
affichage(XGauss_Siedel)
XSteepest_descent = Steepest_descent(A,b,xini)
print "Méthode de plus profonde descente (avec",len(XSteepest_descent[:,0])-1,"itérations):"
affichage(XSteepest_descent)
XConjugate_Gradient = Conjugate_Gradient(A,b,xini)
print "Méthode du gradient conjugué (avec",len(XConjugate_Gradient[:,0])-1,"itérations):"
affichage(XConjugate_Gradient)

## 2. Le problème de Poisson

On s'intéresse maintenant au problème de Poisson en dimension $1$ suivant

$$\left\{\begin{array}{ll} -\Delta u(\ell) = v(\ell), \quad \forall \ell\in ]0,1[, \\ u(0) = u(1) = 0\end{array}\right., $$

où $v$ est une fonction que l'on choisira comme étant

$$ v(\ell) = \left\{\begin{array}{ll} 0 \mbox{ pour } \ell\in[0,1/3[\cap]2/3,1], \\ 1 \mbox{ pour } \ell\in [1/3,2/3]. \end{array}\right.$$

On discrétise l'opérateur de Laplace par des différences finies du second ordre de pas $h = \frac 1 {n+1}$ (avec conditions aux bords de Dirichlet) et l'on en déduit la matrice $A\in M_{n,n}(\mathbb{R})$ associée

$$ A =\left(\begin{array}{ccccc} 2 & -1 & 0 & \dots & 0 \\ -1 & 2 & -1 & \ddots & \vdots \\ 0 & \ddots & \ddots & \ddots & 0\\ \vdots & \ddots & -1 & 2  & -1 \\ 0 & \dots & 0 & -1 & 2\end{array}\right). $$

et donc le système linéaire

$$ Ax = b, $$

où $b\in\mathbb{R}^n$ est donné par

$$ b_i = \left\{ \begin{array}{ll} h^2 \mbox{ pour } i\in \left[\frac{n+1}3,\frac{2(n+1)}3\right] \\ 0 \mbox{ sinon.} \end{array}\right.$$

### 2.0. Définition de la matrice A et du vecteur b

In [ ]:
n = 100
x = np.linspace(0,1,n+2)
h = x[1] - x[0]
def v(x):
    v = np.zeros(len(x))
    for i in range(len(x)):
        if (x[i]>1./3) and (x[i]<2./3):
            v[i] = 1
    return v
def Poisson_discrete(n,v):
    A = np.diag(2*np.ones(n),0)-np.diag(np.ones(n-1),1)-np.diag(np.ones(n-1),-1)
    x = np.linspace(0,1,n+2)
    h = x[1] - x[0]
    b = h**2*v(x[1:n+1])
    return A,b
A,b = Poisson_discrete(n,v)

### 2.1. Résolution du système par la méthode np.solve

Par la suite, on considérera (à raison?) que la solution exacte $x$ du problème est celle donnée par la méthode $np.solve$ de Numpy.

In [ ]:
u = np.zeros(n+2)
u[1:n+1] = npl.solve(A,b)
plt.plot(x,u)
plt.show()

### 2.2. Comparaison des méthodes

On se propose de comparer les méthodes en traçant l'évolution des quantités suivantes au fur et à mesure des itérations:
- la norme $A$ de l'erreur $$e_m(A) = \|x_m-x\|_{A} = \langle x_m-x,A(x_m-x)\rangle^{1/2},$$
- la norme euclidienne de l'erreur $$e_m(2) =\|x_m-x\|_2,$$
- et la norme euclidienne du résidu $$e_m(R) = \|r_m\|_2.$$

> **A faire :** Tracer l'évolution de $e_m(A)$, $e_m(2)$ et $e_m(R)$ pour la méthode de Jacobi, Gauss-Siedel, plus profonde descente et le gradient conjugué en partant du point $x_0 = 0$. Commenter les résultats.

In [ ]:
XJacobi = Jacobi(A,b)
XGauss_Siedel = Gauss_Siedel(A,b)
XSteepest_descent = Steepest_descent(A,b)
XConjugate_Gradient = Conjugate_Gradient(A,b)

In [ ]:
def Error_A(Xiter,x,A):
    E = np.zeros(len(Xiter[:,0]))
    for i in range(len(Xiter[:,0])):
        E[i] = np.dot(Xiter[i,:] - x,np.dot(A,Xiter[i,:] - x))
    return E
def Error_2(Xiter,x,A):
    E = np.zeros(len(Xiter[:,0]))
    for i in range(len(Xiter[:,0])):
        E[i] = npl.norm(Xiter[i,:] - x)
    return E
def Error_R(Xiter,b,A):
    E = np.zeros(len(Xiter[:,0]))
    for i in range(len(Xiter[:,0])):
        E[i] = npl.norm(b - np.dot(A,Xiter[i,:]))
    return E
X = npl.solve(A,b)

EJacobi_A = Error_A(XJacobi,X,A)
EJacobi_2 = Error_2(XJacobi,X,A)
EJacobi_R = Error_R(XJacobi,b,A)
EGauss_Siedel_A = Error_A(XGauss_Siedel,X,A)
EGauss_Siedel_2 = Error_2(XGauss_Siedel,X,A)
EGauss_Siedel_R = Error_R(XGauss_Siedel,b,A)
ESteepest_descent_A = Error_A(XSteepest_descent,X,A)
ESteepest_descent_2 = Error_2(XSteepest_descent,X,A)
ESteepest_descent_R = Error_R(XSteepest_descent,b,A)
EConjugate_Gradient_A = Error_A(XConjugate_Gradient,X,A)
EConjugate_Gradient_2 = Error_2(XConjugate_Gradient,X,A)
EConjugate_Gradient_R = Error_R(XConjugate_Gradient,b,A)

plt.plot(np.log10(EJacobi_A), label = 'E(A)')
plt.plot(np.log10(EJacobi_2), label = 'E(2)')
plt.plot(np.log10(EJacobi_R), label = 'E(R)')
plt.title('Methode de Jacobi')
plt.legend()
plt.show()

plt.plot(np.log10(EGauss_Siedel_A), label = 'E(A)')
plt.plot(np.log10(EGauss_Siedel_2), label = 'E(2)')
plt.plot(np.log10(EGauss_Siedel_R), label = 'E(R)')
plt.title('Methode de Gauss-Siedel')
plt.legend()
plt.show()

plt.plot(np.log10(ESteepest_descent_A), label = 'E(A)')
plt.plot(np.log10(ESteepest_descent_2), label = 'E(2)')
plt.plot(np.log10(ESteepest_descent_R), label = 'E(R)')
plt.title('Methode de plus profonde descente')
plt.legend()
plt.show()

plt.plot(np.log10(EConjugate_Gradient_A), label = 'E(A)')
plt.plot(np.log10(EConjugate_Gradient_2), label = 'E(2)')
plt.plot(np.log10(EConjugate_Gradient_R), label = 'E(R)')
plt.title('Methode du gradient conjugue')
plt.legend()
plt.show()